In [1]:
import pandas as pd
import plotly.graph_objects as go # pip install plotly
import numpy as np
from plotly.graph_objs import *
from scipy import stats
import scipy as sp

In [2]:
ECG_Features = pd.read_csv("DREAMER_ECG.csv")
# Number of features : Nf = 30
Nf=ECG_Features.shape[1]-1
ECG_Features.head()

,Unnamed: 0,ECG_Rate_Mean,HRV_RMSSD,HRV_MeanNN,HRV_SDNN,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,...,HRV_LFn,HRV_HFn,HRV_LnHF,HRV_SD1,HRV_SD2,HRV_SD2SD1,HRV_CSI,HRV_CVI,HRV_CSI_Modified,HRV_SampEn
0,0,1.019883,0.937463,0.980715,0.942895,0.933189,0.961436,0.955897,0.981730,1.000000,...,NaN,NaN,NaN,0.933189,0.933189,1.0,1.0,0.978786,0.933189,0.858731
1,1,0.901621,1.646394,1.108991,1.212197,1.641699,1.093063,1.484588,1.113345,1.104762,...,NaN,NaN,NaN,1.641699,1.641699,1.0,1.0,1.172813,1.641699,1.351139
2,2,1.033613,0.973727,0.967888,1.094067,0.968126,1.130365,1.006033,0.980392,1.111111,...,NaN,NaN,NaN,0.968126,0.968126,1.0,1.0,0.990580,0.968126,0.853926
3,3,0.951886,1.235442,1.049724,1.190338,1.236088,1.133953,1.176921,1.056122,1.142857,...,NaN,NaN,NaN,1.236088,1.236088,1.0,1.0,1.067677,1.236088,0.804541
4,4,1.063608,0.747780,0.942383,0.685394,0.745794,0.727298,0.793499,0.933333,0.786765,...,NaN,NaN,NaN,0.745794,0.745794,1.0,1.0,0.916515,0.745794,1.301240


In [3]:
"""
# List for 3D visualization : aborted !
ECG_Feature_Value=[]
for k in range(0,23):
    for j in range(0,18):
        for f in range(0,Nf):
            if not(np.isnan(ECG_Features.iloc[18*k+j,1+f]) or np.isinf(ECG_Features.iloc[18*k+j,1+f])):
                ECG_Feature_Value.append(ECG_Features.iloc[18*k+j,1+f])
            else:
                ECG_Feature_Value.append(0)
"""

'\n# List for 3D visualization : aborted !\nECG_Feature_Value=[]\nfor k in range(0,23):\n    for j in range(0,18):\n        for f in range(0,Nf):\n            if not(np.isnan(ECG_Features.iloc[18*k+j,1+f]) or np.isinf(ECG_Features.iloc[18*k+j,1+f])):\n                ECG_Feature_Value.append(ECG_Features.iloc[18*k+j,1+f])\n            else:\n                ECG_Feature_Value.append(0)\n'

In [4]:
"""
m=np.min(ECG_Feature_Value)
M=np.max(ECG_Feature_Value)
Colors=[]
Sizes=[]
Color_Amplification=10
for c in range(0,len(ECG_Feature_Value)):
    Colors.append('rgb('+str(round(255*Color_Amplification*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+')')
    if not(ECG_Feature_Value[c]==0):
        Sizes.append(7)
    else:
        Sizes.append(0)
"""

"\nm=np.min(ECG_Feature_Value)\nM=np.max(ECG_Feature_Value)\nColors=[]\nSizes=[]\nColor_Amplification=10\nfor c in range(0,len(ECG_Feature_Value)):\n    Colors.append('rgb('+str(round(255*Color_Amplification*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+','+str(round(255*(ECG_Feature_Value[c]-m)/(M-m),1))+')')\n    if not(ECG_Feature_Value[c]==0):\n        Sizes.append(7)\n    else:\n        Sizes.append(0)\n"

In [5]:
"""
# 3D Visualization : aborted !
fig = go.Figure(data=go.Scatter3d(
    x = np.tile([c for c in range(0,23)],18*Nf)
    ,y = np.tile([np.tile(c,Nf) for c in range(0,18)],23).reshape(18*23*Nf)#np.tile([c for c in range(0,18)],23*Nf)
    ,z = np.tile([c for c in range(0,Nf)],18*23)
    #text = planets,
    ,mode = 'markers',
    marker = dict(
        sizemode = 'diameter',
        size = Sizes, #ECG_Feature_Value,
        color = Colors#['rgb(0,0,255)' for c in range(0,18*23*Nf)],
        )
))
fig.update_layout(
    title="Bubble Plot for the ECG features",
    scene=Scene(
        xaxis=XAxis(title='Subject number'),
        yaxis=YAxis(title='Video number'),
        zaxis=ZAxis(title='Feature number')
    )
    #zaxis_title="Feature number",
    #font=dict(
    #    family="Courier New, monospace",
    #    size=18,
    #    color="#7f7f7f")
    )
fig.show()
"""

'\n# 3D Visualization : aborted !\nfig = go.Figure(data=go.Scatter3d(\n    x = np.tile([c for c in range(0,23)],18*Nf)\n    ,y = np.tile([np.tile(c,Nf) for c in range(0,18)],23).reshape(18*23*Nf)#np.tile([c for c in range(0,18)],23*Nf)\n    ,z = np.tile([c for c in range(0,Nf)],18*23)\n    #text = planets,\n    ,mode = \'markers\',\n    marker = dict(\n        sizemode = \'diameter\',\n        size = Sizes, #ECG_Feature_Value,\n        color = Colors#[\'rgb(0,0,255)\' for c in range(0,18*23*Nf)],\n        )\n))\nfig.update_layout(\n    title="Bubble Plot for the ECG features",\n    scene=Scene(\n        xaxis=XAxis(title=\'Subject number\'),\n        yaxis=YAxis(title=\'Video number\'),\n        zaxis=ZAxis(title=\'Feature number\')\n    )\n    #zaxis_title="Feature number",\n    #font=dict(\n    #    family="Courier New, monospace",\n    #    size=18,\n    #    color="#7f7f7f")\n    )\nfig.show()\n'

In [6]:
from ipywidgets import widgets
import plotly.express as px

ECG_Feature_Widget = widgets.Dropdown(
    options=list(ECG_Features.columns[1:Nf]),
    value='ECG_Rate_Mean',
    description='Feature :',
)

#graphic = go.Histogram(x=ECG_Features['ECG_Rate_Mean'], opacity=0.75, name='Some data')
data=ECG_Features['ECG_Rate_Mean']
graphic = go.Histogram(x=data, opacity=0.75, name='Some data')
g = go.FigureWidget(data=[graphic],
                    layout=go.Layout(
                        title=dict(
                            text='Interactive visualization of the ECG Features (Histogram)'
                        ),
                        barmode='overlay',
                        xaxis=dict(title = ''), yaxis=dict(title = 'Counts[#]')
                    ))

def validate():
    if ECG_Feature_Widget.value in ECG_Features[1:Nf].unique():
        return True
    else:
        return False


def response(change):
            g.data[0].x = ECG_Features[ECG_Feature_Widget.value]
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = ''
            g.layout.yaxis.title = 'Counts[#]'


ECG_Feature_Widget.observe(response, names="value")

container = widgets.HBox([ECG_Feature_Widget])
widgets.VBox([container,g])

In [7]:
# Heatmap

ECG_Feature_Widget2 = widgets.Dropdown(
    options=list(ECG_Features.columns[1:Nf]),
    value='ECG_Rate_Mean',
    description='Feature :',
)
graphic2 = go.Heatmap(z=data.values.reshape(23,18),
                   y=[str('s'+str(k+1)) for k in range(0,23)],
                   x=[str('v'+str(j+1)) for j in range(0,18)])
g2 = go.FigureWidget(data=[graphic2],
                    layout=go.Layout(
                        title=dict(
                            text='Interactive visualization of the ECG Features (Heatmap)'
                        ),
                        barmode='overlay',
                        xaxis=dict(title = 'Video number'), yaxis=dict(title = 'Subject number')
                    ))

def validate2():
    if ECG_Feature_Widget2.value in ECG_Features[1:Nf].unique():
        return True
    else:
        return False


def response2(change):
            g2.data[0].z = ECG_Features[ECG_Feature_Widget2.value].values.reshape(23,18)
            g2.layout.barmode = 'overlay'
            #g2.layout.xaxis.title = ''
            #g2.layout.yaxis.title = 'Counts[#]'


ECG_Feature_Widget2.observe(response2, names="value")

container2 = widgets.HBox([ECG_Feature_Widget2])
widgets.VBox([container2,g2])

In [8]:
All_Features = pd.read_csv("All_Features_with_Age_Gender.csv")
Nf=All_Features.shape[1]-1
All_Features.head()

# Heatmap

All_Feature_Widget = widgets.Dropdown(
    options=list(All_Features.columns[1:Nf]),
    value='psdtheta_1',
    description='Feature :',
)

#graphic = go.Histogram(x=All_Features['All_Rate_Mean'], opacity=0.75, name='Some data')
data=All_Features['psdtheta_1']
graphic = go.Histogram(x=data, opacity=0.75, name='Some data')
g = go.FigureWidget(data=[graphic],
                    layout=go.Layout(
                        title=dict(
                            text='Interactive visualization of the All_Features_with_Age_Gender.csv (Histogram)'
                        ),
                        barmode='overlay',
                        xaxis=dict(title = ''), yaxis=dict(title = 'Counts[#]')
                    ))

def validate():
    if All_Feature_Widget.value in All_Features[1:Nf].unique():
        return True
    else:
        return False


def response(change):
            g.data[0].x = All_Features[All_Feature_Widget.value]
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = ''
            g.layout.yaxis.title = 'Counts[#]'


All_Feature_Widget.observe(response, names="value")

container = widgets.HBox([All_Feature_Widget])
widgets.VBox([container,g])

In [9]:
# Heatmap

All_Feature_Widget2 = widgets.Dropdown(
    options=list(All_Features.columns[1:Nf]),
    value='psdtheta_1',
    description='Feature :',
)
graphic2 = go.Heatmap(z=data.values.reshape(23,18),
                   y=[str('s'+str(k+1)) for k in range(0,23)],
                   x=[str('v'+str(j+1)) for j in range(0,18)])
g2 = go.FigureWidget(data=[graphic2],
                    layout=go.Layout(
                        title=dict(
                            text='Interactive visualization of the All_Features_with_Age_Gender.csv (Heatmap)'
                        ),
                        barmode='overlay',
                        xaxis=dict(title = 'Video number'), yaxis=dict(title = 'Subject number')
                    ))

def validate2():
    if All_Feature_Widget2.value in All_Features[1:Nf].unique():
        return True
    else:
        return False


def response2(change):
            g2.data[0].z = All_Features[All_Feature_Widget2.value].values.reshape(23,18)
            g2.layout.barmode = 'overlay'
            #g2.layout.xaxis.title = ''
            #g2.layout.yaxis.title = 'Counts[#]'


All_Feature_Widget2.observe(response2, names="value")

container2 = widgets.HBox([All_Feature_Widget2])
widgets.VBox([container2,g2])

In [17]:
Valence=All_Features['Valence'].values
Arousal=All_Features['Arousal'].values
Pearson_R=np.empty((2,Nf-4))
p_values=np.empty((2,Nf-4))
for f in range(0,Nf-4):
    if not(All_Features.iloc[:,f].dtype==np.object):
        Pearson_R[0,f], p_values[0,f] = sp.stats.pearsonr(Valence,All_Features.iloc[:,f].values)
        Pearson_R[1,f], p_values[1,f] = sp.stats.pearsonr(Arousal,All_Features.iloc[:,f].values)

heatmap_Pearson_R = px.imshow(Pearson_R,
                y=['Valence','Arousal'],
                x=All_Features.columns[0:Nf-4]#[str('f'+str(j+1)) for j in range(0,Nf-4)]
                )
#heatmap_Pearson_R.update_xaxes(side="top")
heatmap_Pearson_R.show()



In [18]:
heatmap_p_values = px.imshow(p_values,
                y=['Valence','Arousal'],
                x=All_Features.columns[0:Nf-4]#[str('f'+str(j+1)) for j in range(0,Nf-4)]
                )

heatmap_p_values.show()

In [25]:
corrMatrix = All_Features.corr()

heatmap_corrMatrix = px.imshow(corrMatrix,
                               title='Cross-correlation matrix (All_Features_with_Age_Gender.csv)',
                               
                y=All_Features.columns[0:Nf],
                x=All_Features.columns[0:Nf]#[str('f'+str(j+1)) for j in range(0,Nf-4)]
                )

heatmap_corrMatrix.show()